Coursera의 Neural Networks for Machine Learning 강의를 정리한 내용임.   
2017.01.09. by Dongwan Kim

# Week14. Deep neural nets with generative pre-training

### Lecture 14a, Learning layers of features by stacking RBMs

이번 장 에서는 Sigmoid belief net을 학습시키는 다른 방법에 대해서 알아보겠다. 이 방법은 사실 예상치 못하게 발견된 측면이 있는데, 사실 Hinton은 Boltzmann machine에 대한 연구 때문에 Sigmoid Belief net에 대한 연구를 별로 진행하지 않고 있었는데, 이때 Restricted Boltzmann machine을 이용해 nonlinear한 feature들을 쉽게 학습시킬 수 있다는 것을 발견하게 된다. 그렇다면 이렇게 학습된 feature들을 data로 취급하여 추가적인 Restriected Boltzmann machine을 학습시켜 feature들 간의 correlation을 모델링할 수 있지 않을까 하는 아이디어를 갖게 된다. 즉 RBM위에 또 다른 RBM을 쌓는 형식으로 많은 layer의 nonlinear feature을 학습시키겠다는 것이다. 이런 아이디어로 인해 Deep neural net에 대한 연구자들의 관심이 증가하게 된다.

그런데 이 아이디어에서 많은 RMB을 쌓았을 때 이것이 여러개의 독립적인 RBM인 것인지 혹은 이것을 하나의 모델로 간주해야할지가 이슈가 된다. 얼핏 보면 이런 중첩 모델이 하나의 'multi-layer Boltzmann machine'이지 않을까 생각할 수 있는데, Hinton의 제자인 <a href="https://www.stats.ox.ac.uk/~teh/">Yee Whye Teh</a>가 이런 중첩 모델이 하나의 'multi-layer Boltzmann machine'가 아니고 대신 Sigmoid belief net과 유사한 것임을 밝혀 낸다.

이는 Deep sigmoid belief net을 학습시키는 문제를 (우회적으로) Boltzmann machine과 같은 undirected model을 학습시키는 방식으로 해결할 수 있었을 것이라는 것 때문에 Yee Whye Teh의 발견은 Hinton에게는 대단히 놀라운 것이었다. 

<u>Train a deep network by stacking RBMs</u>

어떤 이미지를 입력으로 받는 RBM을 생각해보자. 이 RBM을 Learning 시킬 수 있고(Lecture 12.c 참고) 이렇게 학습된 RBM의 hidden layer activation들을 다른 RBM의 입력으로 사용할 수 있다. 이런 RBM 중첩을 여러번 반복할 수 있고, 각 RMB layer는 이전 RBM의 activation에 존재하는 correlation을 모델링한다고 볼 수 있다. 

우리가 RBM을 중첩할 때마다 이 중첩 모델이 해당 데이터를 generate할 log probability의 variational lower을 개선하는(높이는) 효과가 있다는 것을 증명할 수 있다. 증명은 사실 좀 복잡한데, RBM이 결국 infinitely deep belief net와 같다는 사실을 기반으로 한다.)

<img  src="./_images/12_rbm1.PNG"/>

<u>Combining two RBMs to make a DBN</u>

간단한 예로 두개의 RMB을 중첩해보고 이것이 결국 하나의 Deep Belief net이 되는 과정을 살펴보자.

우선 아래 좌측 하단과 같이 하나의 RBM에 data가 주어졌을 때 이를 학습시킨다. 그리고 이때 hidden layer($h_1$)의 activation vector를 두번째 RBM(아래 좌측 상단)의 입력으로 사용하여 두번째 RBM을 학습시킨다.

그런데 흥미로운 점은 두번째 RBM의 hidden layer $h_2$의 unit 수가 첫번째 RBM의 visible layer $v$의 unit수 만큼이면, 두번째 RBM의 weight matrix($W_2$) 자리에 첫번째 RBM의 weight matrix($W_1$)의 transpose를 사용하면 이미 두번째 RBM이 $h_1$에 대한 좋은 모델이 된다는 것이다. 이것은 두번째 RBM이 첫번째 RBM을 뒤집은 형태이기 때문이다. 

이제 두 RBM을 하나로 합치면 아래 우측의 모형이 되는데, 상단의 두 layer $h_2$와 $h_1$은 좌측 상단의 RBM와 같다고 할 수 있다. 즉 symmetric weight을 사용하는 undirected graph이다. 반면 하단의 두 layer는 조금 다르다. 이 경우 sigmoid belief net과 같은 directed graph이다. 즉 앞서 좌측 하단 RBM에서 $v$에서 $h_1$으로 향하는 weight을 없앤 형태가 된다. (왜 이렇게 되는지는 상당히 복잡한 설명이 필요하며, Lecture 14e에서 살펴보도록 하겠다.

이렇게 만들어진 우측의 (합쳐진) 모형은 하단에 non symmetric connection을 갖고 있으므로 RBM은 분명 아니다. 이는 Deep belief net이라 부르는 graphical model인데, 하단의 두 layer는 sigmoid belief net이고 상단의 두 layer는 RBM인 형태인 hybrid model이라 할 수 있다.

<img  src="./_images/14_stackingRBM.PNG"/>

<u>The generative model after learning 3 layers</u>

RBM을 3개 쌓을 경우 아래와 같은 hybrid model이 만들어진다. 상단의 두 layer는 RBM이고, 두번째 layer부터는 sigmoid belief net과 같은 directed graph이다.

이 모델에서 data를 generate하기 위해서는 (즉 weight들이 정해져 있고 model에서 데이터 생성)
- 우선 $h2$와 $h3$를 RBM으로 간주하고 이 두 layer를 번갈아 업데이트하여 thermal equilibrium에 이르도록 한다. 이 과정은 alternating gibbs sampling을 이용해 (아마도 한 layer에서 update할 unit을 여러개 선택하는 것으로 보임) $h_2$와 $h_3$의 unit들을 병렬로 update하는 것이다. 사실 이 RBM은 $h_2$에 대한 prior distribution을 만드는 과정이라 할 수 있다.
- 다음으로 앞서 thermal equilibrium에 다다른 RBM의 $h_2$에서 activation vector를 얻고 이를 weight이 $W_2$인 generative connection을 통해 $h_1$으로 보낸다. 그리고 다시 한번 weight이 $W_1$인 generative connection을 통해 data를 생성한다. (사실 $h_2$에서 data까지의 과정은 단순히 sigmoid belief net에서의 그것과 동일하다.)

아래 그림에서 붉은색 화살표는 사실 모델에 포함된 내용이 아니다. 이것들은 각각에 대응하는 녹색 weight matrix의 transpose이다. 이것들은 사실 추론(inference)에 사용되는 것들이다.

<img  src="./_images/14_stacking3RBM.PNG"/>

.

<u>An aside: Averaging factorial distributions</u>

이제 Boltzmann machine을 쌓는 것(stacking up)이 왜 좋은 아이디어 인지 살펴보겠는데,
우선 두개의 factorial distribution을 averaging 하는 것이 어떤 의미인지부터 알아보자. <i>(Lecture13d에서 다뤘다시피 factorial distribution은 서로 독립인 확률변수들(혹은 그중 일부)의 joint probability distribution을 의미한다.)</i>

두개의 factorial distribution을 averaging(taking mixture of distribution)한 distribution이 꼭 factorial distribution이지는 않는다. 예를들어 hidden layer에 4개의 unit이 있는 RBM이 있을 경우 data가 주어졌을 때, 4개의 hidden unit에 대한 posterior distribution은 factorial distribution이다. 어떤 dataset에 대한 posterior disibution이 $v_1$: (0.9, 0.9, 0.1, 0.1)이고 다른 dataset에 대한 posterior distribution이 $v_2$: (0.1, 0.1, 0.9, 0.9)일때 각각 unit probability를 averaging하면 $Aggregated$: (0.5, 0.5, 0.5, 0.5)를 얻게 되는데 이는 factorial distribution일까?

예를들어 hidden layer의 state vector가 (1, 1, 0, 0)일 확률을 살펴보면 $v_1$에서는 그 확률이 $P(1, 1, 0, 0) = 0.9 \times 0.9 \times (1-0.1) \times (1-0.1) = 0.43$이고 $v_2$에서는 그 확률이 $P(1, 1, 0, 0) = 0.1 \times 0.1 \times (1-0.9) \times (1- 0.9) = 0.001$이다. 이 두 확률을 averaging할 경우 $(0.43 + 0.001)/2 = 0.2155$을 얻게 되는데, 이 값은 $Aggregated$가 factorial distribution이라 했을 때 계산될 수 있는 $0.5^4$와 다르다. 즉 이 분포가 factorial distribution이 아니라 단순히 mixture distribution임을 알 수 있다.


<u>Why does greedy learning work?</u>

이제 greedy learning(여러 layer의 RBM을 학븟하는 방식)는 이유에 대해 알아보자. 가장 아래 층 RBM의 weight들은 그것이 연결하는 두개의 layer(hidden, visible)에 대한 분포를 결정한다. 우선 $p(v|h)$와 $p(h|v)$는 hidden layer와 visible layer를 번갈아가며 update하는데 사용될 수 있고 (update을 반복한 후) thermal equilibrium에 이르면 두 layer에서 sampling을 통해 $p(v,h)$또한 구할 수 있다. 또한 $p(v,h)$에서 $h$를 marginalize하면 $p(v)$를 얻게 되고 이것은 visible layer에 대한 prior distribution이며, 마찬가지로 $v$를 marginalize하면 $p(h)$를 얻게 되고 이는 hidden layer에 대한 prior distribution이다. 

RBM 모델이라는 거은 결국 $p(v)$를 최대화 하는 것인데 $p(v)$를 아래와 같이 표현할 수 있다.
$$p(v) = \sum_h p(h) p(v|h)$$
이것이 의미하는 바는 $p(v|h)$는 그대로 두고 $p(h)$를 개선하더라도 결국은 $p(v)$를 개선할 수 있다는 것이다. 여기에서 $p(h)$를 개선한다는 것은 aggregated poseteior distribution에 더 잘 맞는 '$h$에 대한 prior distribution'을 찾는 것이다. 이를 위해 첫번째 RBM이 aggregated posterior을 갖도록 학습시키고, 두번째 RBM이 앞서 aggregated posterior를 더 잘 모델링하도록 학습시킨다. 

<u>Fine-tuning with a contrastive version of the wake-sleep algorithm</u>

이렇게 RBM들을 학습시킨 후 이것들을 중첩시켜 Deep belief net을 만들고, 약간 변형된 wake-sleep algorithm을 이용해 (bottom-up recognition weights(아래 그림에서 빨간색 weight)와 top-down generative weights(아래 그림에서 하단의 녹색 weight)를 update 해서 전체 모델의 성능을 개선시킬 수 있다.

이 fine-tuning은 3단계로 진행되는데
- 우선 가장 하단의 layer부터 data를 흘려보내고(stochastic bottom-up pass), 이때 sampled correleation을 이용해 top-down weight을 조정한다.(wake-sleep algorithm의 wake phase와 같음)
- 가장 상단의 RBM에서 sampling & update(Constrastive Divergence learning)을 반복한다. 
- 방금전 상단 RBM의 visible layer의 activation vector를 이용해 top-down pass를 진행한다.(sigmoid belief net에서 forward pass와 같음) 이때 sampled correlation을 이용해 bottom-up weight을 조정한다.(wake-sleep algorithm의 sleep phase와 같음, 다만 단순히 독립성만을 가정하는 wake-sleep algorithm와 달리 top-level RBM이 훨신 좋은 prior를 갖는다는 것이 다르다.)

<img  src="./_images/14_stacking3RBM.PNG"/>

<u>The DBN used for modeling the joint distribution of MNIST dights and their labels</u>

아래와 같이 MINIST dights와 각각의 labels(0~9)의 joint distribution을 모델링할 수 있었다고 한다.

<img  src="./_images/14_MNIST.PNG"/>

### Lecture 14b, Discriminative fine-tuning for DBMs

이번에는 Boltzmann machine을 중첩하여 deep belief net을 학습시키고나서 어떤 class label간의 판별(distriminatory)을 하는 network로 간주하고 fine-tuning을 하는 것을 살펴보자. 앞서 lecture14a에서는 더 나은 generative model을 만드는 것이 목표 였다면 이번에는 class간 판별을 목적으로 한다.)

이번 방법이 speech recognition에서는 많은 영향을 주었고, 관련된 많은 곳에서 deep neural net으로 그 방법론을 바꿔가고 있다. 

<u>Fine-tuning for discrimination</u>

판별(Discrimination)을 위한 Fine-tuning 방법에 대해 알아보자.
- 우선 RBM하나식 학습하여 중첩시킨다.
- 앞서 학습된 RBM들을 weight 초기값을 설정하기 위한 'pre-training'이라 할 수 있고, local search procedure를 통해 이들을 fine-tune(Constrastive wake-sleep method)하게 된다. 결국 이는 better generative model을 만드는 과정이라 할 수 있다.
- Discrimination을 위해 fine-tune 과정에 Backpropagation을 사용할 수 있다. (앞서 pre-training을 거치고 나면 backpropagation의 많은 제약사항을 극복할 수 있고, deep net에 대해서도 사용이 가능해진다. 또한 network의 generalization 또한 좋아진다.)

<u>Why backpropagation works better with greedy pre-training: The optimization view</u>

그렇다면 weight들을 pre-train하는 것이 왜 backpropagation에 좋은 영향을 주는 것일까? 이는 두가지 관점에서 살펴볼 수 있는데 우선 최적화(optimization)관점에서 살펴보자. 각 layer 단위의 RBM learning(greedily learning)을 하게 되면 weight 들의 scale이 나아진다. 특히 locality가 있는 network의 경우 효과적일 것이다.
또한 feature detector를 거치고 나서(RBM training) backpropagation을 사용하게 되므로 discrimination에 유의미한 feature들이 이미 구분되어 있을 것이라 생각할 수 있다. 따라서 backpropagation은 이미 어느정도 정리가 된? sensible starting point로부터 단지 local search만을 하는 것으로 충분한 효과를 보이게 된다.(global search를 할 필요가 없음)

<u>Why backpropagation works better with greedy pre-training: the overfitting view</u>

최종 weight에 포함되는 대부분의 정보는 input vector의 분포로부터 나온다고 할 수 있는데, 이런 input vector는 일반적으로 label보다 훨신 많은 정보를 포함하고 있다. 예를들어 image 판별 문제에서 image pixel들이 담고 있는 정보는 그 이미지가 무엇인지에 관한 label보다 훨신 많은 정보를 담고 있다. 그런데 'greedy pre-training'을 할 경우 label에 관한 정보는 오직 fine-tuning과정에서만 사용된다.(소중한 label 정보가 낭비되지 않음)

또한 이미 feature들이 다 찾아졌으므로 fine-tuning은 단지 category 경계를 잘 나누기 위해 feature들을 조금 수정하게 된다. 즉 backpropagation을 통해 새로운 feature를 찾을 필요가 없어진다.

이런 형태의 backpropagation은 training data에 unlabeled data가 많다 하더라도 잘 동작하는 장점을 갖는다. 이는 pre-training 과정은 unlabeled data로 충분하기 때문이다.

이런 pre-training 방법에 대한 반론으로, 필요 없는 feature들 까지도 pre-training에 포함되는 점을 지적할 수 있다. 하지만 이런 문제는 computing power가 좋아지면서 큰 이슈가 아니라고 할 수 있다.

<u>First, model the distribution of digit images</u>

앞서 설명한 방법을 MNIST 손글씨 데이터에 적용한 모델이 아래와 같다.

<img  src="./_images/14_MNIST_discriminate.PNG"/>

.

<u>Results on the permutation-invariant MNIST task</u>

<img  src="./_images/14_a1.PNG"/>

<u>Unsupervised "pre-training" also helps for models that have more data and better priors</u>

<img  src="./_images/14_a2.PNG"/>

<u>Phone recognition on the TIMIT benchmark</u>

<img  src="./_images/14_a3.PNG"/>

### Lecture 14c, What happens during discriminative fine-tuning?

pre-training 이후 fine-tuning과정에서 어떤 일이 일어나는지 살펴볼텐데, 결론적으로 lower layer의 weight들은 fine-tune 과정에서 거의 변하지 않는데, 이런 작은 변화가 decision boundary를 움직이므로 classification performance에 큰 영향을 주게 된다. 

또한 효과적인 pre-training을 사용할 경우 deeper network가 shallow network보다 더 나은 성능을 보이게 된다. (pre-training이 없을 경우 deeper network의 성능이 꼭 더 좋은 것이 아니다.)

<u>Learning Dynamics of Deep Nets</u>

아래 그림은 요슈아 벤지오(Yoshua Bengio)'s group에서 연구된 내용으로, fine-tuning 과정에서 어떤 일이 일어나는지를 보여준다. 아래 그림은 feature detector의 첫번째 hidden layer의 receptive fields들로써 fine-tuning 이전과 이후 큰 차이가 없음을 알 수 있다. 하지만 이런 약간의 변화가 discrimination 성능에는 큰 영향을 주고 있다. 

<img  src="./_images/14_bengio1.PNG"/>

<u>Effect of Unsupervised Pre-training</u>

아래 그림은 pre-training이 test error에 주는 영향을 보여준다.
좌측 그림은 hidden layer가 하나만 있는 network에서 fine-tuning 이후 test error를 보여주는데, pretraining을 한 경우(빨간색)에 항상 더 낮은 test error를 보여줌을 알 수 있다. 우측 그림을 보면 hidden layer가 4개인 network에서의 실험인데 pretraining의 효과가 더 커짐을 알 수 있다. 즉 network가 깊어질 수록 pretraining이 보다 더 효과적이라 할 수 있다. 또한 pretraining이 없을 경우에는 deep network의 성능이 shallow network보다 좋지 않음도 알 수 있다. 

<img  src="./_images/14_beingio2.PNG"/>

<u>Effect of Depth</u>

아래 그림은 layer의 수가 늘어날 때 그리고 pre-training 사용했을 때와 그렇지 않을 때 test classification error를 보여준다. 
우선 좌측 그림을 pre-training이 없을 경우인데, 차례대로 hidden layer가 1, 2, 3, 4개인 경우를 보여준다. layer가 많아질 경우 error가 증가함을 알 수 있다. 
반면 우측 그림은 pre-training을 사용한 경우인데, 전반적으로 pre-tranining이 없을 때 보다 error가 낮아졌고, layer가 많아질 경우 error가 더 작아짐을 알 수 있다.

<img  src="./_images/14_bengio3.PNG"/>

<u>Trajectories of the learning in function space</u>

아래 그림은 t-sne를 이용해 function space를 모형화 한 것인데, 각 점이 하나의 모형이고 training 과정에서 function의 궤적을 확인 할 수 있다. 또한  색은 iteration(epoch)의 수를 의미하는데, 파란 점에서 시작하여 점차 노란 빨간 점으로 퍼져나가고 있다.

우측 상단의 점 들은 pre-training을 사용하지 않은 경우인데, 파란 점에서 시작하여 점차 멀리 퍼져 감을 알 수 있다. 반면 하단의 점들은 pre-training을 사용한 경우인데 여러 epoch이후에도 점들이 (비교적) 크게 퍼지지 않음을 알 수 있다. 그런데 이 두 점그룹 사이에 overlap이 없다는 것이 흥미로운데, pre-training을 할 경우 질적으로 다른 (더 나은) solution을 얻게 된다는 것이다.

<img  src="./_images/14_t-sne1.PNG"/>

<u>Why unsupervised pre-training makes sense</u>

조금 다른 관점에서 pre-training이 효과적인 이유를 살펴보자.

우선 왼쪽 그림과 같이 stuff(실체)가 image를 결정하고 이 image가 label을 결정한다면, image를 이용해 바로 label을 추론하는 것이 합리적이라고 할 수 있다. (즉 label이 그 실체에 독립적인 경우를 말함, 물론 그럴리 없음)

아래 우측의 그림은 좀더 그럴듯한 상관관계인데, 어떤 stuff(실체)가 image와 label에 모두 영향을 주는 관계이다. 즉 image가 label을 결정하는 것이 아니라 stuff가 image와 label모두를 결정한다는 것이다. (이 관계가 보다 합리적으로 보인다.) 그런데 이 관계에서는 stuff에 대한 image는 많은 정보(high bandwidth)를 담게 된다. 반면 stuff에 대한 label은 적은 정보(low bandwidth)만을 담게 된다. 예를 들어 어떤 동물이 한마리 있고 이것이 '강아지'라는 사실은 label인데, 이 정보만으로는 이 강아지의 색이나 크기 등은 알 수 없다. 반면 image는 강아지에 대한 많은 정보를 담게 된다. 

우측과 같은 관계를 상정하면 image로부터 어떤 실체(stuff)에 관한 지식을 얻고 이를 이용해 label을 추론하는 것이 합리적이라고 할 수 있다. 이때 image로부터 어떤 지식을 얻는 과정을 pre-training을 통해 진행된다고 할 수 있다. 

<img  src="./_images/14_stuffs.PNG"/>

### Lecture 14d, Modeling real-valued data with an RBM

이번에는 RBM을 이용해 real-valued data를 모델링하는 방법에 대해 알아보자. 즉 visible unit이 앞서 binary(0 or 1) value가 아니라 실수값인 경우이다. visible unit이 real-value일 때에는 어떻게 학습을 진행할지가 문제가 되는데, 이에 대한 해결책으로 hidden layer에 **RELU, Rectified Linear Unit**을 사용하는 방법을 사용할 수 있다.

Linear Gaussian unit을 visible layer에 배치하고 Rectified Linear Unit을 hidden layer에 배치하면 RBM을 학습시키는 것이 쉬워진다.

<u>Modeling real-valued data</u>

우선 RBM을 이용해 hand-written digit을 판별하는 문제를 생각해 볼 텐데, 이미지에서 어떤 픽셀의 색은 잉크가 전혀 뭍지 않았거나 완전히 검게 뭍은 경우 혹은 옅게 뭍은 경우가 있을 것이다. 이때 잉크가 뭍은 정도를 mean-field logistic unit을 사용해 0과 1 사이의 확률로 나타내는 것을 생각해볼 수 있다. 

하지만 이런 접근 법은 실제 image 판별 문제에는 효과적이지 않다. 실제 이미지에서는 특정 pixel의 intensity가 거의 항상 주변 pixel의 intensity의 평균으로 결정되기 때문이다. (매우 작은 확률로 평균 값과 다른 intensity가 발생한다.) 즉 이런 경우 logistic unit을 이용해서는 좋은 결과를 얻기 어렵다. 

<u>A standard type of real-valued visible unit</u>

앞서 문제에 대한 대안으로 hidden layer에 Gaussian noise가 포함된 linear unit을 사용해 볼 수 있다. 각 pixel을 Gaussian variable로 모델링하고 Alternating Gibbs sampling을 사용한다. 이때 매우 작은 learning rate을 사용해야 한다. 그렇지 않을 경우 blow up하고 만다.

이때의 energy function은 아래와 같다.



<img  src="./_images/14_real_valued_energy2.PNG"/>

첫번째 term은 visible layer가 발생시키는 energy로서 우측 상단 그림의 2차원 곡선이 된다. 이때 energy를 최소화 하기 위해서는 visible unit의 값이 bias에 가까워지게 되는 특성이 있음을 알 수 있다. 반면 3번째 term은 visible-hidden unit의 interaction으로 발생하는 energy로서 우측 상단 그림의 파란색 직선이 된다. 

이 둘을 더할 경우 전체 energy minimum은 2차원 곡선의 bias가 한쪽으로 이동된 위치에 형성될텐데, 이 위치의 shifting은 파란 직선의 기울기에 따라 결정된다.

<u>Gaussian-Binary RBM's</u>

이런 형태의 energy model을 학습시키는 문제가 쉽지 않은데 그 이유는 아래 그림으로 설명 된다. bottom-up effect는 variance에 반비례하고 top-down effect는 variance에 비례하기 때문에, variance가 작은 경우 bottom-up effect는 지나치게 커져서 hidden unit의 값이 양 극단에 몰리게 되고 top down effect는 지나치게 작아진다.

<img  src="./_images/14_sigma_problem.PNG"/>

$$E(\mathbf{v}, \mathbf{h}) = \sum_{i \in vis} \frac{{(v_i - b_i)}^2}{2 \sigma_i^2} - \sum_{j \in hid} b_j h_j - \sum_{i, j} \frac{v_i}{\sigma_i} h_j w_{ij}$$


<u>Stepped sigmoid units: A neat way to implement integer values</u>

앞서의 문제를 해결하기 위한 아이디어로 Stepped sigmoid unit을 사용하는 방법이 있다.      
hidden layer의 stochastic binary unit을 여러개 복사하고 각 (복사항)(hidden) unit과 visible unit간의 weight을 같게 한다. 그리고 bias를 일정한 step을 갖도록 달리한다. 이럴 경우 variance가 작아지면 켜지는 hidden unit의 수가 많아지고 그만큼 top-down effect또한 커지게 된다.(앞서 top-down effect가 작아지는 문제를 해결)

<img  src="./_images/14_bias_step.PNG"/>

<u>Fast approximations</u>

그런데 앞서의 해결책은 binary stochastic unit이 지나치게 많아지는 문제점을 갖는다. 이에 대한 해결책으로 근사 방법을 사용할 수 있다. 

bias가 다른 여러개의 binary stochastic unit들의 activation 합을 아래와 같이 근사할 수 있다. 

<img  src="./_images/14_approxi.PNG"/>

Contrstive Divergence는 위 첫번째 식을 이용한 경우에도 잘 동작하는데 이때의 noise variance는 $sigmoid(\sum_n y)$$와 같다. 물론 Contrstive Divergence는 위 3번째 식을 이용한 경우(**RELU**, rectified linear units)에도 잘 동작하고 computation speed 또한 매우 우수하다.

<u>A nice property of rectified linear units</u>

RELU(Rectified Linear Unit)의 좋은 특성중 하나는 bias가 0인 경우 scale equivariant하다는 것이다. 이런 특성을 image 데이터를 처리할 때 특히 유용한데, 그 관계를 아래와 같이 나타낼 수 있다.

즉 어떤 image 데이터를 matrix $x$라 할 때, 모든 pixel intensity에 어떤 상수 $a$를 곱하 image를 function input으로 하는 RELU값은 원본 이미지릐 RELU 출력에 $a$를 곱한 것과 같다는 것이다. (이미지를 scale up하는 것은  모든 visible unit의 activity를 동일한 값만큼 scale up한 것과 같다는 것이다.) 하지만 RELU는 어떤 두 이미지의 합을 넣은 출력 값이 각 이미지를 넣은 출력값의 합과 같지 않기 때문에 완전히 linear하지는 않다. 


<img  src="./_images/14_relu_property.PNG"/>

RELU의 이런 특성은 Convolution net의 translation equivariance와 유사한데, pulling을 무시할 경우 아래와 같이 shift 이후의 출력값과 출력값을 shift한 결과가 같다.

<img  src="./_images/14_covlu.PNG"/>

### Lecture 14e, RBMs are Infinite Sigmoid Belief Nets (ADVANCED)

<u>Another view of why layer-by-layer learning works</u>

.

<u>An infinite sigmoid belief net that is equivalent to an RBM</u>

.

<u>Inference in an infinite sigmoid belief net</u>

.

<u>Learning a deep directed network</u>

.

<u>What happens when the weights in higher layers become different from the weights in the first layer?</u>

.

<u>What is really happening in contrastive divergence learning?</u>

.

<u>Why is it OK to ignore the derivatives in higher layers?</u>

.